<a href="https://colab.research.google.com/github/fauzanismara/penelitian3/blob/main/Pewarnaan_VGG16(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================
# 1. Mount Google Drive
# =========================================
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# =========================================
# 2. Import library
# =========================================
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# =========================================
# 3. Bagi dataset otomatis (train/val/test)
# =========================================
import os
import shutil
import random
import imghdr

source_dir = "/content/gdrive/MyDrive/data_penelitian/Pewarnaan"  # folder awal
destination_dir = "/content/gdrive/MyDrive/dataset_Pewarnaan_Split"

train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2

# Buat folder train/val/test
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(destination_dir, split), exist_ok=True)

# Loop setiap kelas
classes = os.listdir(source_dir)
for cls in classes:
    cls_source = os.path.join(source_dir, cls)
    if not os.path.isdir(cls_source):
        continue  # skip kalau bukan folder kelas

    print(f"Memproses kelas: {cls}")

    # Ambil semua file gambar dari subfolder juga
    all_images = []
    for root, _, files in os.walk(cls_source):
        for f in files:
            fpath = os.path.join(root, f)
            # hanya ambil gambar valid
            if os.path.isfile(fpath) and imghdr.what(fpath) is not None:
                all_images.append(fpath)

    # Acak urutan gambar
    random.shuffle(all_images)

    # Hitung jumlah untuk tiap split
    n_total = len(all_images)
    n_train = int(train_ratio * n_total)
    n_val = int(val_ratio * n_total)
    # sisanya untuk test
    n_test = n_total - n_train - n_val

    # Bagi dataset
    train_files = all_images[:n_train]
    val_files = all_images[n_train:n_train+n_val]
    test_files = all_images[n_train+n_val:]

    splits = {"train": train_files, "val": val_files, "test": test_files}

    # Copy ke folder tujuan
    for split_name, file_list in splits.items():
        split_cls_dir = os.path.join(destination_dir, split_name, cls)
        os.makedirs(split_cls_dir, exist_ok=True)

        for src_path in file_list:
            fname = os.path.basename(src_path)
            dst_path = os.path.join(split_cls_dir, fname)
            shutil.copy(src_path, dst_path)

    print(f"Kelas {cls}: {n_total} gambar → train={len(train_files)}, val={len(val_files)}, test={len(test_files)}")

print("✅ Dataset selesai di-split!")




Memproses kelas: Dedak 100_
Kelas Dedak 100_: 200 gambar → train=140, val=20, test=40
Memproses kelas: Dedak 90_
Kelas Dedak 90_: 200 gambar → train=140, val=20, test=40
Memproses kelas: Dedak 60_
Kelas Dedak 60_: 200 gambar → train=140, val=20, test=40
Memproses kelas: Sekam 100_
Kelas Sekam 100_: 200 gambar → train=140, val=20, test=40
Memproses kelas: Dedak 70_
Kelas Dedak 70_: 200 gambar → train=140, val=20, test=40
Memproses kelas: Dedak 80_
Kelas Dedak 80_: 200 gambar → train=140, val=20, test=40
✅ Dataset selesai di-split!


In [ ]:
import os, glob
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Path dataset hasil split
dataset_dir = "/content/gdrive/MyDrive/dataset_Pewarnaan_Split"
train_dir = os.path.join(dataset_dir, "train")
val_dir = os.path.join(dataset_dir, "val")
test_dir = os.path.join(dataset_dir, "test")

# Data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

num_classes = len(train_gen.class_indices)

# ======================
# 1. Bangun model VGG16
# ======================
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze semua layer dulu
for layer in base_model.layers:
    layer.trainable = False

# Buka (unfreeze) block5 (layer terakhir)
for layer in base_model.layers[-4:]:
    layer.trainable = True

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=preds)

# Pakai optimizer dengan learning rate kecil biar fine-tuning stabil
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# ======================
# 2. Training
# ======================
history = model.fit(
    train_gen,
    epochs=20,   # bisa ditambah
    validation_data=val_gen
)

# ======================
# 3. Evaluasi di test set
# ======================
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

# ======================
# 4. Visualisasi hasil training
# ======================
plt.figure(figsize=(12,5))

# akurasi
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

# ======================
# 5. Confusion Matrix & Report
# ======================
y_pred = model.predict(test_gen)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_gen.classes
class_labels = list(test_gen.class_indices.keys())

cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.title("Confusion Matrix")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

print("Classification Report:\n")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))


Found 840 images belonging to 6 classes.
Found 120 images belonging to 6 classes.
Found 240 images belonging to 6 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,139,014 (80.64 MB)

 Trainable params: 13,503,750 (51.51 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1060s 39s/step - accuracy: 0.1812 - loss: 2.0727 - val_accuracy: 0.4167 - val_loss: 1.6839
Epoch 2/20
 7/27 ━━━━━━━━━━━━━━━━━━━━ 10:50 33s/step - accuracy: 0.1948 - loss: 1.8890